## **Quiz ML No. 2**

---

### Logistic Regression

Nama  : William Yulio

NIM   : 2301869840

Kelas : LD01

### Import Library yang akan digunakan

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix

### Membaca Data Input dari CSV

In [2]:
data = pd.read_csv('credit-approval_csv.csv')
data

,Gender,Age,Debt,Married,BankCustomer,EducationLevel,Ethnicity,YearsEmployed,PriorDefault,Employed,CreditScore,DriversLicense,Citizen,ZipCode,Income,Approved,Approved_Status
0,b,30.83,0.000,u,g,w,v,1.25,t,t,1,f,g,202.0,0,+,Approved
1,a,58.67,4.460,u,g,q,h,3.04,t,t,6,f,g,43.0,560,+,Approved
2,a,24.50,0.500,u,g,q,h,1.50,t,f,0,f,g,280.0,824,+,Approved
3,b,27.83,1.540,u,g,w,v,3.75,t,t,5,t,g,100.0,3,+,Approved
4,b,20.17,5.625,u,g,w,v,1.71,t,f,0,f,s,120.0,0,+,Approved
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
685,b,21.08,10.085,y,p,e,h,1.25,f,f,0,f,g,260.0,0,-,Not Approved
686,a,22.67,0.750,u,g,c,v,2.00,f,t,2,t,g,200.0,394,-,Not Approved
687,a,25.25,13.500,y,p,ff,ff,2.00,f,t,1,t,g,200.0,1,-,Not Approved
688,b,17.92,0.205,u,g,aa,v,0.04,f,f,0,f,g,280.0,750,-,Not Approved


### Data Cleaning 

In [3]:
data = data.replace({"?":np.nan})

# Mengganti data-data dengan value '?' menjadi null, agar bisa dimasukkan value baru kedalamnya

In [4]:
data.fillna(data.mean(), inplace=True)
data.isnull().sum()

# Melakukan impute kedalam missing value yang baru saja kita proses berdasarkan mean dari datasetnya
# Selanjutkan kita akan melakukan check apakah masih terdapat data null didalam dataset lainnya

Gender             12
Age                 0
Debt                0
Married             6
BankCustomer        6
EducationLevel      9
Ethnicity           9
YearsEmployed       0
PriorDefault        0
Employed            0
CreditScore         0
DriversLicense      0
Citizen             0
ZipCode             0
Income              0
Approved            0
Approved_Status     0
dtype: int64

In [5]:
for col in data:
    
    if (data[col].dtypes == 'object'):
        data = data.fillna(data[col].value_counts().index[0])
        
data.isnull().sum()

# Selanjutnya kita beralih dari kolom scalar menuju ke kolom categorical
# Didalam for ini kita akan melakukan checking apakah terdapat data yang memiliki value null untuk kolom categorical
# Kemudian diisi dengan data yang paling sering muncul

Gender             0
Age                0
Debt               0
Married            0
BankCustomer       0
EducationLevel     0
Ethnicity          0
YearsEmployed      0
PriorDefault       0
Employed           0
CreditScore        0
DriversLicense     0
Citizen            0
ZipCode            0
Income             0
Approved           0
Approved_Status    0
dtype: int64

### Encode data kategorial menjadi scalar value

In [6]:
Encoder = LabelEncoder()
data['Approved_Status'] = Encoder.fit_transform(data['Approved_Status'])

data.head()

# melakukan encode terhadap output yang berupa approved dan not approved kedalam bentuk scalar agar lebih mudah untuk diolah

,Gender,Age,Debt,Married,BankCustomer,EducationLevel,Ethnicity,YearsEmployed,PriorDefault,Employed,CreditScore,DriversLicense,Citizen,ZipCode,Income,Approved,Approved_Status
0,b,30.83,0.000,u,g,w,v,1.25,t,t,1,f,g,202.0,0,+,0
1,a,58.67,4.460,u,g,q,h,3.04,t,t,6,f,g,43.0,560,+,0
2,a,24.50,0.500,u,g,q,h,1.50,t,f,0,f,g,280.0,824,+,0
3,b,27.83,1.540,u,g,w,v,3.75,t,t,5,t,g,100.0,3,+,0
4,b,20.17,5.625,u,g,w,v,1.71,t,f,0,f,s,120.0,0,+,0


In [7]:
data_kategori = data[['Gender', 'Married', 'BankCustomer', 'EducationLevel', 'Ethnicity', 'PriorDefault', 'Employed', 'DriversLicense', 'Citizen']]

data_kategori = data_kategori.apply(Encoder.fit_transform)

# melakukan encode terhadap data yang bersifat kategori sehingga bisa memiliki value scalar

In [8]:
data = data.drop(data_kategori.columns, axis=1)
data = pd.concat([data, data_kategori], axis=1)

# Kemudian menggabungkan kembali data_kategori yang sudah di encode sebelumnya kedalam dataset yang utama

In [9]:
Correlation = data.corr()
Correlation['Approved_Status'].abs().sort_values(ascending = False)

# Selanjutnya kita akan melihat variabel mana yang memiliki korelasi tertinggi dengan variabel output
# Kita akan melakukan proses seleksi variabel bebas mana yang akan dimasukkan kedalam model
# Variabel bebas yang akan diambil merupakan variabel yang memiliki korelasi >= 0.1
# Variabel yang dibawah 0.1 akan didrop dari tabel dataset

Approved_Status    1.000000
PriorDefault       0.720407
Employed           0.458301
CreditScore        0.406410
YearsEmployed      0.322475
Debt               0.206294
Married            0.191431
BankCustomer       0.187520
Income             0.175657
Age                0.161627
EducationLevel     0.130026
Citizen            0.100867
ZipCode            0.099972
DriversLicense     0.031625
Gender             0.028934
Ethnicity          0.000877
Name: Approved_Status, dtype: float64

In [10]:
data = data.drop(['Ethnicity','Gender','DriversLicense','ZipCode'], axis=1)
data.head()

# Kemudian variabel yang sebelumnya sudah kita lihat korelasinya dengan variabel independent
# akan didrop, sehingga kita hanya akan menggunakan variabel dengan korelasi yang tinggi saja

,Age,Debt,YearsEmployed,CreditScore,Income,Approved,Approved_Status,Married,BankCustomer,EducationLevel,PriorDefault,Employed,Citizen
0,30.83,0.000,1.25,1,0,+,0,2,1,13,1,1,0
1,58.67,4.460,3.04,6,560,+,0,2,1,11,1,1,0
2,24.50,0.500,1.50,0,824,+,0,2,1,11,1,0,0
3,27.83,1.540,3.75,5,3,+,0,2,1,13,1,1,0
4,20.17,5.625,1.71,0,0,+,0,2,1,13,1,0,2


### Melakukan pemetaan terhadap data input dan output kedalam variabel x dan y 

In [11]:
X = data[['Debt','YearsEmployed','CreditScore','Income','Age','Married','BankCustomer','PriorDefault','Employed','Citizen','EducationLevel']]    # independent varible
y = data[['Approved_Status']]     # dependent variable

X.head()

,Debt,YearsEmployed,CreditScore,Income,Age,Married,BankCustomer,PriorDefault,Employed,Citizen,EducationLevel
0,0.000,1.25,1,0,30.83,2,1,1,1,0,13
1,4.460,3.04,6,560,58.67,2,1,1,1,0,11
2,0.500,1.50,0,824,24.50,2,1,1,0,0,11
3,1.540,3.75,5,3,27.83,2,1,1,1,0,13
4,5.625,1.71,0,0,20.17,2,1,1,0,2,13


### Melakukan Standard Scaler terhadap dataset

In [12]:
sc = StandardScaler()
X = sc.fit_transform(X)
print(X[:10])

# Pada tahap ini kita akan berfokus untuk melakukan preprocessing sehingga dataset menjadi lebih mudah untuk diolah
# menggunakan fungsi StandardSCaler() 

[[-0.95661321 -0.29108264 -0.28810053 -0.19541334 -0.0623209  -0.4529545
  -0.54294996  0.95465038  1.15714435 -0.31717105  1.44996197]
 [-0.06005053  0.24419044  0.74082993 -0.08785188  2.28810134 -0.4529545
  -0.54294996  0.95465038  1.15714435 -0.31717105  0.99633524]
 [-0.8561017  -0.21632383 -0.49388662 -0.03714433 -0.59673802 -0.4529545
  -0.54294996  0.95465038 -0.86419641 -0.31717105  0.99633524]
 [-0.64703776  0.45650546  0.53504384 -0.19483712 -0.31559915 -0.4529545
  -0.54294996  0.95465038  1.15714435 -0.31717105  1.44996197]
 [ 0.17414129 -0.15352643 -0.49388662 -0.19541334 -0.96230297 -0.4529545
  -0.54294996  0.95465038 -0.86419641  3.27050148  1.44996197]
 [-0.15252112  0.08271141 -0.49388662 -0.19541334  0.04321171 -0.4529545
  -0.54294996  0.95465038 -0.86419641 -0.31717105  0.76952187]
 [-0.74754927  1.27885237 -0.49388662  5.81362283  0.13523615 -0.4529545
  -0.54294996  0.95465038 -0.86419641 -0.31717105  1.22314861]
 [ 1.37223851 -0.65291528 -0.49388662  0.0636945

### Melakukan Split Dataset

In [13]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Kita akan melakukan pemisahan dataset utuh kedalam variabel untuk melakukan testing dan juga training
# Dimana data yang akan digunakan untuk test sebesar 20% dari total data utuhnya dan 80% lainnya dimasukkan kedalam variabel training

### Membuat Model Logistic Regression

In [14]:
regressor = LogisticRegression()
regressor.fit(x_train, y_train)

# Memasukkan data yang sudah kita split sebelumnya kedalam model linear regression

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

### Kalkulasi Predict Value

In [15]:
y_pred = regressor.predict(x_test)
y_pred

# Melakukan kalkulasi terhadap predict value menggunakan model yang sudah kita buat sebelumnya

array([1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1,
       0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1,
       0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0,
       1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1,
       1, 0, 1, 1, 0, 0])

### Membuat Confusion Matrix

In [17]:
print("Accuracy of logistic regression : ", regressor.score(x_test,y_test))

# Print the confusion matrix of the logreg model
print("Confusion Matrix \n", confusion_matrix(y_test, y_pred))

# Menghitung akurasi dan confusion matrix untuk melihat seberapa akurat model yang sudah kita buat sebelumnya
# Pada model ini, saya mendapatkan akurasi yang cukup baik yaitu berupa 0.90 dibandingkan dengan jawaban saya pada nomor 1

Accuracy of logistic regression :  0.9057971014492754
Confusion Matrix 
 [[62  2]
 [11 63]]
